<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/10_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Check ENG

In [ ]:
col_dict = {
    'site_value': 'col6',
    'lcl': 'col7',
    'ucl': 'col8',
    'group_lst': "['col3', 'col4']", # 'col3'
}
def check_ENG(df, col_dict):
  df[['site_value', 'lcl', 'ucl']] = df[['site_value', 'lcl', 'ucl']].astype(float)
  # Step 1: 新增 'Site_fail' 欄位
  df['Site_fail'] = df.apply(lambda row: 0 if row['col7'] < row['col6'] < row['col8'] else 1, axis=1)

  # Step 2: 以 col3 和 col4 為分群條件(PRODWFR_ID)，計算 'Site_fail' 的比例
  eng_rate = df.groupby(['col3', 'col4'])['Site_fail'].apply(lambda x: (1 - x).mean()).reset_index()
  eng_rate = eng_rate.rename(columns={'Site_fail': 'ENG_Rate'})

  # Step 3: 使用 left join 將 'ENG_Rate' 合併回原始資料
  df = df.merge(eng_rate, on=['col3', 'col4'], how='left')

  # Step 4: 根據 'ENG_Rate' 與 'col5' 的比較新增 'ENG_fail' 欄位
  df['ENG_fail'] = df.apply(lambda row: 0 if row['ENG_Rate'] > row['col5'] else 1, axis=1)

  return df

## ENG rule

In [ ]:
def check_rule_ENG(df, col_dict):
  result_df = check_ENG(df, col_dict)
  result_df = result_df[result_df['ENG_fail']==1] # df[(df['ENG_fail']==1)&(df['Site_fail']==1)]

  return result_df


In [ ]:
# Step 1: 新增 'Site_fail' 欄位
df['Site_fail'] = df.apply(lambda row: 0 if row['col7'] < row['col6'] < row['col8'] else 1, axis=1)

# Step 2: 以 col3 和 col4 為分群條件，計算 'Site_fail' 的比例
eng_rate = df.groupby(['col3', 'col4'])['Site_fail'].apply(lambda x: (1 - x).mean()).reset_index()
eng_rate = eng_rate.rename(columns={'Site_fail': 'ENG_Rate'})

# Step 3: 使用 left join 將 'ENG_Rate' 合併回原始資料
df = df.merge(eng_rate, on=['col3', 'col4'], how='left')

# Step 4: 根據 'ENG_Rate' 與 'col5' 的比較新增 'ENG_fail' 欄位
df['ENG_fail'] = df.apply(lambda row: 0 if row['ENG_Rate'] > row['col5'] else 1, axis=1)

# Step 5: 回傳資料 (先篩選出 ENG_fail = 1的)
df = df[df['ENG_fail']==1] # 回傳該lot下 fail 的 wafer_id (含所有site資料)
df = df[(df['ENG_fail']==1)&(df['Site_fail']==1)] # 回傳該lot下 fail 的 wafer_id 下 site_fail=1的部分


## Site Rule

In [ ]:
def check_rule_site(df, col_dict):
  result_df = check_ENG(df, col_dict)
  result_df = result_df[result_df['ENG_fail']==0]
  # 接下來check 有無三片wafer fail在同 site_id
  .....

In [ ]:
# Step 1: 新增 'Site_fail' 欄位
df['Site_fail'] = df.apply(lambda row: 0 if row['col7'] < row['col6'] < row['col8'] else 1, axis=1)

# Step 2: 以 col3 和 col4 為分群條件，計算 'Site_fail' 的比例
eng_rate = df.groupby(['col3', 'col4'])['Site_fail'].apply(lambda x: (1 - x).mean()).reset_index()
eng_rate = eng_rate.rename(columns={'Site_fail': 'ENG_Rate'})

# Step 3: 使用 left join 將 'ENG_Rate' 合併回原始資料
df = df.merge(eng_rate, on=['col3', 'col4'], how='left')

# Step 4: 根據 'ENG_Rate' 與 'col5' 的比較新增 'ENG_fail' 欄位
df['ENG_fail'] = df.apply(lambda row: 0 if row['ENG_Rate'] > row['col5'] else 1, axis=1)

# Step 5: 保留 ENG_fail = 0的項目，再去做site_fail check，最後回傳資料記得清除後面新增的col(保留raw data)

## WAT Rule

## 彙總 : 所有結果

(groupby LOT, CATE) (若該rule有trigger項則在此row標記為1)

### 1. 一次 groupby

In [ ]:
import pandas as pd

# 建立範例資料
data = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C'],
    'col2': [1, 1, 1, 2, 2, 2, 3, 3],
    'col3': ['X', 'X', 'Y', 'X', 'Y', 'Y', 'X', 'X'],
    'col4': ['P', 'P', 'P', 'Q', 'Q', 'Q', 'P', 'P'],
    'value': [5, 10, 15, 20, 25, 30, 35, 40]
}
df = pd.DataFrame(data)

# 定義多個函數，模擬不同的計算
def fun1(group):
    return group['value'].mean()  # 計算平均值

def fun2(group):
    return group['value'].sum()   # 計算總和

def fun3(group):
    return group['value'].max()   # 找出最大值

def fun4(group):
    return group['value'].min()   # 找出最小值

def fun5(group):
    return group['value'].std()   # 計算標準差

# 建立一個列表來儲存結果
results = []

# 使用 groupby 進行分組並逐一套用函數
for key, group_data in df.groupby(['col1', 'col2', 'col3', 'col4']):
    result1 = fun1(group_data)
    result2 = fun2(group_data)
    result3 = fun3(group_data)
    result4 = fun4(group_data)
    result5 = fun5(group_data)

    # 將結果與對應的群組標記一起儲存
    results.append({
        'col1': key[0],
        'col2': key[1],
        'col3': key[2],
        'col4': key[3],
        'mean': result1,
        'sum': result2,
        'max': result3,
        'min': result4,
        'std': result5
    })

# 將結果轉為 DataFrame 方便查看
results_df = pd.DataFrame(results)

# 顯示結果
print(results_df)


### 2.

In [ ]:
import pandas as pd

# 建立範例資料
data = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C'],
    'col2': [1, 1, 1, 2, 2, 2, 3, 3],
    'col3': ['X', 'X', 'Y', 'X', 'Y', 'Y', 'X', 'X'],
    'col4': ['P', 'P', 'P', 'Q', 'Q', 'Q', 'P', 'P'],
    'value': [5, 10, 15, 20, 25, 30, 35, 40]
}
df = pd.DataFrame(data)

# 定義多個函數，模擬不同的計算
def fun1(group):
    return group['value'].mean()

def fun2(group):
    return group['value'].sum()

def fun3(group):
    return group['value'].max()

def fun4(group):
    return group['value'].min()

def fun5(group):
    return group['value'].std()

# Step 1: 根據 col1, col2, col3, col4 分群，計算指標並儲存結果
group_results = []

for key, group_data in df.groupby(['col1', 'col2', 'col3', 'col4']):
    result1 = fun1(group_data)
    result2 = fun2(group_data)
    result3 = fun3(group_data)
    result4 = fun4(group_data)
    result5 = fun5(group_data)

    group_results.append({
        'col1': key[0],
        'col2': key[1],
        'col3': key[2],
        'col4': key[3],
        'mean': result1,
        'sum': result2,
        'max': result3,
        'min': result4,
        'std': result5
    })

group_results_df = pd.DataFrame(group_results)
print("各群 (col1, col2, col3, col4) 的結果：")
print(group_results_df)

# Step 2: 根據 col1, col2, col3 彙總，計算平均發生率、總和等
summary_results = group_results_df.groupby(['col1', 'col2', 'col3']).agg(
    avg_mean=('mean', 'mean'),
    total_sum=('sum', 'sum'),
    max_value=('max', 'max'),
    min_value=('min', 'min'),
    std_dev=('std', 'mean')  # 平均標準差
).reset_index()

print("\n根據 col1, col2, col3 的彙總結果：")
print(summary_results)
